In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## 0. Making the artificial data

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
np.random.seed(1234)

In [ ]:
n_dim = 2
x_train, y_train = make_blobs(n_samples=100, n_features=n_dim, centers=[[1,1],[-1,-1]], shuffle=True)
x_test, y_test = make_blobs(n_samples=30, n_features=n_dim, centers=[[1,1],[-1,-1]], shuffle=True)

In [ ]:
print(x_train[:10,:])
print(y_train[:10])

In [ ]:
# Training data
plt.figure()
plt.scatter(x_train[(y_train==0),0], x_train[(y_train==0),1], c='blue', marker='o')
plt.scatter(x_train[(y_train==1),0], x_train[(y_train==1),1], c='red', marker='o')

In [ ]:
# Test data
plt.figure()
plt.scatter(x_test[(y_test==0),0], x_test[(y_test==0),1], c='blue', marker='+')
plt.scatter(x_test[(y_test==1),0], x_test[(y_test==1),1], c='red', marker='+')

In [ ]:
# Training data + Test data
plt.figure()
plt.scatter(x_train[(y_train==0),0], x_train[(y_train==0),1], c='blue', marker='o')
plt.scatter(x_train[(y_train==1),0], x_train[(y_train==1),1], c='red', marker='o')
plt.scatter(x_test[(y_test==0),0], x_test[(y_test==0),1], c='blue', marker='+')
plt.scatter(x_test[(y_test==1),0], x_test[(y_test==1),1], c='red', marker='+')

## 1. Logistic regression

In [ ]:
# Logistic function (a.k.a., softmax function, sigmoid function)
def logistic(x):
    return 1.0 / (1 + np.exp(-x))

In [ ]:
def logit(x, beta):
    logit = np.dot(x, beta)
    return logit

In [ ]:
beta = [0.1, -0.1]
y_hat = logistic(logit(x=x_train, beta=beta)) # Y = 1일 확률
print(y_hat[:10])
print(y_train[:10])

In [ ]:
def negative_log_likelihood(x, y, beta, avg=False):
    y_hat = logistic(logit(x=x_train, beta=beta))
    log_likelihood = y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)
    if not avg:
        return -log_likelihood
    else:
        return np.mean(-log_likelihood)

In [ ]:
loss = negative_log_likelihood(x=x_train, y=y_train, beta=beta)
print(loss)

## 2. Learning logistic regression model
- Using batch gradient descent method

In [ ]:
def logistic_regression(x, y, num_steps=50, learning_rate=0.01):
    initial_beta = np.asarray([10, 10])
    beta_list = []
    beta_list.append(initial_beta)
    beta = initial_beta
    
    for step in range(num_steps):
        y_hat = logistic(logit(x=x_train, beta=beta))
        gradient = np.dot(x.T, y_hat - y)
        new_beta = beta - learning_rate * gradient
        loss = negative_log_likelihood(x, y, beta, avg=True)
        
        # Print
        loss_tracking = '[%d step] loss %.4f' % (step, loss)
        beta_equation = '[%.2f %.2f] = [%.2f %.2f] - %.2f * [%.2f %.2f]' % (new_beta[0], new_beta[1], beta[0], beta[1], learning_rate, gradient[0], gradient[1])
        print(loss_tracking, '\t', beta_equation)
        
        beta_list.append(new_beta)
        beta = new_beta
        
    return beta, beta_list

In [ ]:
beta, beta_list = logistic_regression(x=x_train, y=y_train, num_steps=100, learning_rate=0.01)

In [ ]:
print(beta)
print(beta_list)

In [ ]:
# Comes from Kyunghyun Cho's ML lecture (https://github.com/nyu-dl/Intro_to_ML_Lecture_Note)
def vis_hyperplane(beta, style='k--'):

    lim0 = plt.gca().get_xlim()
    lim1 = plt.gca().get_ylim()
    m0, m1 = lim0[0], lim0[1]

    intercept0 = -(beta[0] * m0 + beta[-1])/beta[1]
    intercept1 = -(beta[0] * m1 + beta[-1])/beta[1]
    
    plt1, = plt.plot([m0, m1], [intercept0, intercept1], style)

    plt.gca().set_xlim(lim0)
    plt.gca().set_ylim(lim1)
        
    return plt1

In [ ]:
# Training data + Hyperplane
plt.figure()
plt.scatter(x_train[(y_train==0),0], x_train[(y_train==0),1], c='blue', marker='o')
plt.scatter(x_train[(y_train==1),0], x_train[(y_train==1),1], c='red', marker='o')

initial_beta_plot = vis_hyperplane(beta_list[0], 'k--')
beta_plot_10 = vis_hyperplane(beta_list[9], 'b--')
beta_plot_15 = vis_hyperplane(beta_list[14], 'r--')
beta_plot = vis_hyperplane(beta, 'g--')

plt.legend([initial_beta_plot, beta_plot_10, beta_plot_15, beta_plot], ['Randomly initialized beta', 'Learned beta(10th updated)', 'Learned beta(15th updated)', 'Learned beta'])

In [ ]:
# Test data + Hyperplane
plt.figure()
plt.scatter(x_test[(y_test==0),0], x_test[(y_test==0),1], c='blue', marker='+')
plt.scatter(x_test[(y_test==1),0], x_test[(y_test==1),1], c='red', marker='+')

initial_beta_plot = vis_hyperplane(beta_list[0], 'k--')
beta_plot = vis_hyperplane(beta, 'g--')

plt.legend([initial_beta_plot, beta_plot], ['Randomly initialized beta', 'Learned beta'])

In [ ]:
# Training data + Test data
plt.figure()
plt.scatter(x_train[(y_train==0),0], x_train[(y_train==0),1], c='blue', marker='o', alpha=0.1)
plt.scatter(x_train[(y_train==1),0], x_train[(y_train==1),1], c='red', marker='o', alpha=0.1)
plt.scatter(x_test[(y_test==0),0], x_test[(y_test==0),1], c='blue', marker='+')
plt.scatter(x_test[(y_test==1),0], x_test[(y_test==1),1], c='red', marker='+')

initial_beta_plot = vis_hyperplane(beta_list[0], 'k--')
beta_plot = vis_hyperplane(beta, 'g--')

plt.legend([initial_beta_plot, beta_plot], ['Randomly initialized beta', 'Learned beta'])